In [ ]:
year = 2023
surface = "Hard"
opponent = "Alcaraz C."  # or "All"

# Filter data
dff = df[(df["Year"] == year) & (df["Surface"].str.lower() == surface.lower())]
if opponent != "All":
    dff = dff[dff["Top10"] == opponent]

# Collect traces
players = {}
for _, row in dff.iterrows():
    name = row["Underdog"]
    if name not in players:
        players[name] = {"ranks": [], "odds": []}
    players[name]["ranks"].append(row["UDRank"])
    players[name]["odds"].append(row["Odds"])

traces = []
for player, stats in players.items():
    avg_rank = sum(stats["ranks"]) / len(stats["ranks"])
    avg_return = sum(stats["odds"]) / len(stats["odds"])
    traces.append(go.Scatter(
        x=[avg_rank],
        y=[avg_return],
        mode="markers+text",
        name=player,
        text=[player],
        textposition="top center",
        marker=dict(size=20, opacity=0.8),
        hovertemplate=f"{player}<br>Avg Rank: {avg_rank:.1f}<br>Avg Return: ${avg_return:.2f}<extra></extra>"
    ))

# Render only selected plot
fig = go.Figure(data=traces)
fig.update_layout(
    title=f"ATP Underdog Returns vs {opponent} in {year} on {surface}",
    xaxis=dict(title="Average Rank", autorange="reversed"),
    yaxis=dict(title="Avg Return ($1 Stake)", rangemode="tozero"),
    font=dict(family="Arial", size=14),
    height=700,
    paper_bgcolor="#fff",
    plot_bgcolor="#f9f9f9"
)

fig.write_html("clean_static_plot.html", include_plotlyjs="cdn", full_html=True)


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from itertools import product

# Load data
df = pd.read_json("docs/assets/data/underdogs.json")

# Unique options
years = sorted(df["Year"].unique())
surfaces = sorted(df["Surface"].str.capitalize().unique())

# Build list of (year, surface, opponent) triples
combinations = []
traces_dict = {}

for year, surface in product(years, surfaces):
    top10s = sorted(df[(df["Year"] == year) & (df["Surface"].str.lower() == surface.lower())]["Top10"].dropna().unique())
    top10s = ["All"] + top10s
    for opponent in top10s:
        key = (year, surface, opponent)
        dff = df[(df["Year"] == year) & (df["Surface"].str.lower() == surface.lower())]
        if opponent != "All":
            dff = dff[dff["Top10"] == opponent]
        players = {}
        for _, row in dff.iterrows():
            name = row["Underdog"]
            if name not in players:
                players[name] = {"ranks": [], "odds": []}
            players[name]["ranks"].append(row["UDRank"])
            players[name]["odds"].append(row["Odds"])
        traces = []
        for player, stats in players.items():
            avg_rank = sum(stats["ranks"]) / len(stats["ranks"])
            avg_return = sum(stats["odds"]) / len(stats["odds"])
            traces.append(go.Scatter(
                x=[avg_rank],
                y=[avg_return],
                mode="markers+text",
                name=player,
                text=[player],
                textposition="top center",
                marker=dict(size=20, opacity=0.8),
                hovertemplate=f"{player}<br>Avg Rank: {avg_rank:.1f}<br>Avg Return: ${avg_return:.2f}<extra></extra>",
                visible=False
            ))
        combinations.append(key)
        traces_dict[key] = traces

# Flatten all traces
all_traces = [trace for group in traces_dict.values() for trace in group]
fig = go.Figure(data=all_traces)

# Helper to get visibility mask
def get_visibility(target_key):
    vis = [False] * len(all_traces)
    start = 0
    for key in combinations:
        traces = traces_dict[key]
        if key == target_key:
            for i in range(len(traces)):
                vis[start + i] = True
        start += len(traces)
    return vis

# Initial combo
init_year = years[-1]
init_surface = "Hard"
init_opponent = "All"
init_key = (init_year, init_surface, init_opponent)
init_vis = get_visibility(init_key)

# Buttons
year_buttons = []
for y in years:
    year_buttons.append(dict(
        label=str(y),
        method="update",
        args=[
            {"visible": get_visibility((y, init_surface, init_opponent))},
            {"title": f"ATP Underdog Returns vs {init_opponent} in {y} on {init_surface}"}
        ]
    ))

surface_buttons = []
for s in surfaces:
    surface_buttons.append(dict(
        label=s,
        method="update",
        args=[
            {"visible": get_visibility((init_year, s, init_opponent))},
            {"title": f"ATP Underdog Returns vs {init_opponent} in {init_year} on {s}"}
        ]
    ))

# Unique opponent buttons (must match specific year/surface pairs)
opponent_buttons = []
for (y, s, o) in combinations:
    if y == init_year and s == init_surface:
        opponent_buttons.append(dict(
            label=o,
            method="update",
            args=[
                {"visible": get_visibility((y, s, o))},
                {"title": f"ATP Underdog Returns vs {o} in {y} on {s}"}
            ]
        ))

# Layout
fig.update_layout(
    title={
        "text": f"WTA Underdog Returns vs {init_opponent} in {init_year} on {init_surface}",
        "x": 0.5,
        "xanchor": "center",
        "font": dict(size=22)
    },
    updatemenus=[
        dict(
            buttons=year_buttons,
            direction="down",
            x=0.1, y=1.2,
            xanchor="left",
            yanchor="top",
            showactive=True
        ),
        dict(
            buttons=surface_buttons,
            direction="down",
            x=0.4, y=1.2,
            xanchor="left",
            yanchor="top",
            showactive=True
        ),
        dict(
            buttons=opponent_buttons,
            direction="down",
            x=0.7, y=1.2,
            xanchor="left",
            yanchor="top",
            showactive=True
        ),
    ],
    xaxis=dict(
        title="Average Rank",
        titlefont=dict(size=16),
        tickfont=dict(size=14),
        autorange="reversed"
    ),
    yaxis=dict(
        title="Avg Return ($1 Stake)",
        titlefont=dict(size=16),
        tickfont=dict(size=14),
        rangemode="tozero"
    ),
    font=dict(family="Arial", size=14, color="#111"),
    legend=dict(
        title="Underdogs",
        font=dict(size=13),
        orientation="v",
        x=1.02,
        y=1,
        xanchor="left"
    ),
    margin=dict(l=60, r=120, t=100, b=60),
    height=700,
    paper_bgcolor="#ffffff",
    plot_bgcolor="#f9f9f9"
)


# Set initial visibility
for i, v in enumerate(init_vis):
    fig.data[i].visible = v

# Save
fig.write_html("interactive_wta_underdog_final.html", auto_open=True)

